In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.activations import *
from keras.layers import *
from keras.losses import *
from keras.optimizers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import talos as ta
%matplotlib inline

np.set_printoptions(suppress=True)

Using TensorFlow backend.


In [2]:
path = (r'C:\Users\MacalusoC\Desktop\test.csv')

data = pd.read_csv(path)

In [3]:
# scale the data between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))

# scale the data
scaled_data = scaler.fit_transform(data)

In [4]:
# separate np array into feature and label data
scaled_features = scaled_data[:, [3, 4]]
scaled_labels = scaled_data[:, [0, 1, 2]]

# split the data into train and test groups
train_features, test_features, train_labels, test_labels = train_test_split(
    scaled_features, scaled_labels, test_size=0.2, random_state=0)

In [5]:
p = {
    'first_neuron': [8, 12, 24],
    'second_neuron': [36, 48, 62, 84],
    'third_neuron': [36, 48, 62, 84],
    'fourth_neuron': [8, 12, 24],
    'batch_size': [10, 20, 30],
    'activation': [relu, 
                   softmax,
                   selu,
                   softplus,
                   softsign,
                   tanh,
                   sigmoid,
                   hard_sigmoid,
                   exponential,
                   linear],
    'optimizer' : ['SGD',
                   'RMSprop',
                   'Adagrad',
                   'Adadelta',
                   'Adam',
                   'Adamax',
                   'Nadam',],
    'loss' : [mean_squared_error, 
              mean_absolute_error, 
              mean_absolute_percentage_error, 
              mean_squared_logarithmic_error,
              squared_hinge,hinge,
              categorical_hinge,
              logcosh,],
}

In [6]:
def ann(train_features, train_labels, test_features, test_labels, params):
    
    # replace the hyperparameter inputs with references to params dictionary 
    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=2, activation=params['activation']))
    model.add(Dense(params['second_neuron'], activation=params['activation']))
    model.add(Dense(params['third_neuron'], activation=params['activation']))
    model.add(Dense(params['fourth_neuron'], activation=params['activation']))
    model.add(Dense(3, activation=params['activation']))
    model.compile(loss=params['loss'], optimizer=params['optimizer'], metrics=['accuracy'])
    
    # make sure history object is returned by model.fit()
    out = model.fit(train_features, train_labels,
                    epochs=100,
                    batch_size=params['batch_size'],
                    # validation_split=.3,
                    verbose=0,
                    validation_data=[test_features, test_labels])
    
    # modify the output model
    return out, model

In [7]:
# run the Talos experiment
t = ta.Scan(scaled_features, scaled_labels, 
            params=p, 
            model=ann,
            grid_downsample=.01,
            )

  0%|                                                                                         | 0/2419 [00:00<?, ?it/s]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


100%|██████████████████████████████████████████████████████████████████████████████| 2419/2419 [46:39<00:00,  1.38s/it]


In [8]:
# accessing the results data frame
t.data.head()

# accessing epoch entropy values for each round
t.peak_epochs_df

# access the summary details
t.details

random_method          uniform_mersenne
reduction_method                   None
reduction_interval                   50
reduction_window                     20
grid_downsample                    0.01
reduction_threshold                 0.2
reduction_metric                val_acc
reduce_loss                       False
experiment_name           031919145539_
complete_time            03/19/19/15:42
x_shape                         (18, 2)
y_shape                         (18, 3)
dtype: object

In [9]:
# use Scan object as input
r = ta.Reporting(t)

In [14]:
# get the number of rounds in the Scan
r.rounds()

# get correlation for hyperparameters against a metric
#r.correlate('acc')

2419

In [15]:
# get the best paramaters
r.best_params()

array([['24', '30', '12', 'Adamax', '48',
        '<function linear at 0x0000019FEBABE510>', '36', 0],
       ['24', '20', '12', 'RMSprop', '84',
        '<function selu at 0x0000019FEBABE0D0>', '84', 1],
       ['8', '20', '24', 'SGD', '62',
        '<function tanh at 0x0000019FEBABE2F0>', '84', 2],
       ['24', '30', '12', 'Adam', '62',
        '<function linear at 0x0000019FEBABE510>', '84', 3],
       ['8', '10', '24', 'Adamax', '62',
        '<function selu at 0x0000019FEBABE0D0>', '84', 4],
       ['12', '10', '8', 'SGD', '48',
        '<function linear at 0x0000019FEBABE510>', '36', 5],
       ['8', '10', '8', 'Adadelta', '62',
        '<function tanh at 0x0000019FEBABE2F0>', '36', 6],
       ['24', '10', '24', 'Adam', '48',
        '<function linear at 0x0000019FEBABE510>', '62', 7],
       ['24', '30', '24', 'Adagrad', '84',
        '<function tanh at 0x0000019FEBABE2F0>', '84', 8],
       ['12', '10', '24', 'Adadelta', '62',
        '<function tanh at 0x0000019FEBABE2F0>', '

In [16]:
# get the round with the best result
r.rounds2high()

28

In [20]:
# get the highest result ('val_acc' by default)
r.high('val_acc')

'1.0'

In [18]:
# get the highest result for any metric
r.high('acc')

'0.8333333432674408'

In [22]:
e = ta.Evaluate(t)
e.evaluate(scaled_features, scaled_labels, folds=10)

[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]